In [37]:
import os
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup

DETAIL_SELECTOR = "div.articleContent"
PARTIAL_CSV = "nstc_jobs_partial.csv"
FINAL_CSV = "nstc_jobs_full.csv"
HEADERS = {"User-Agent": "Mozilla/5.0"}

# Step 1: 讀取主清單（你已經有了）
url = "https://www.nstc.gov.tw/folksonomy/list/ba3d22f3-96fd-4adf-a078-91a05b8f0166?l=ch&pageSize=500&pageNum=1"
res = requests.get(url, headers=HEADERS)
res.encoding = "utf-8"
soup = BeautifulSoup(res.text, "html.parser")

records = []
for a_tag in soup.select("a:has(h3)"):
    title = a_tag.h3.text.strip()
    link = "https://www.nstc.gov.tw" + a_tag.get("href")
    date_tag = a_tag.select_one("div.date")
    date = date_tag.text.strip() if date_tag else ""
    records.append({"職缺名稱": title, "發佈日期": date, "連結": link})

df_master = pd.DataFrame(records)

# Step 2: 讀已爬取資料（如果有的話）
if os.path.exists(PARTIAL_CSV):
    df_done = pd.read_csv(PARTIAL_CSV)
    done_links = set(df_done["連結"])
    print(f"✅ 已完成 {len(done_links)} 筆，將略過")
else:
    df_done = pd.DataFrame(columns=["職缺名稱", "發佈日期", "連結", "詳細內容"])
    done_links = set()

# Step 3: 開始爬取未完成的詳細內容
for i, row in df_master.iterrows():
    if row["連結"] in done_links:
        continue

    try:
        r = requests.get(row["連結"], headers=HEADERS, timeout=15)
        r.encoding = "utf-8"
        soup = BeautifulSoup(r.text, "html.parser")
        content_div = soup.select_one(DETAIL_SELECTOR)
        detail = content_div.get_text(separator="\n", strip=True) if content_div else "（無內容）"

        # 加入新資料
        new_row = {
            "職缺名稱": row["職缺名稱"],
            "發佈日期": row["發佈日期"],
            "連結": row["連結"],
            "詳細內容": detail
        }

        # 立即儲存（append）
        df_done = pd.concat([df_done, pd.DataFrame([new_row])], ignore_index=True)
        df_done.to_csv(PARTIAL_CSV, index=False, encoding="utf-8-sig")
        print(f"✅ 完成：{row['職缺名稱']}")

    except Exception as e:
        print(f"⚠️ 失敗：{row['連結']}，原因：{e}")
        continue

    time.sleep(1)  # 禮貌等待

# Step 4: 最終整理完整檔案
df_done.to_csv(FINAL_CSV, index=False, encoding="utf-8-sig")
print(f"\n📦 所有資料儲存完成，共 {len(df_done)} 筆，檔案：{FINAL_CSV}")


✅ 已完成 19 筆，將略過
⚠️ 失敗：https://www.nstc.gov.twhttps://reurl.cc/0W2D86，原因：HTTPSConnectionPool(host='www.nstc.gov.twhttps', port=443): Max retries exceeded with url: //reurl.cc/0W2D86 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001E975C8F5B0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
⚠️ 失敗：https://www.nstc.gov.twhttps://www.cgu.edu.tw/cmci/Contents?nodeId=16966，原因：HTTPSConnectionPool(host='www.nstc.gov.twhttps', port=443): Max retries exceeded with url: //www.cgu.edu.tw/cmci/Contents?nodeId=16966 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001E975C8F6D0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
⚠️ 失敗：https://www.nstc.gov.twhttps://job.fju.edu.tw/#/vacaDtl/294，原因：HTTPSConnectionPool(host='www.nstc.gov.twhttps', port=443): Max retries exceeded with url: //job.fju.edu.tw/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00

KeyboardInterrupt: 

In [46]:
df_result = []
df_result = pd.read_csv("nstc_jobs_partial.csv")
df_result.head()

,職缺名稱,發佈日期,連結,詳細內容
0,本會網站「求才訊息」免責聲明,2016-08-24,https://www.nstc.gov.tw/folksonomy/detail/b4a0...,(一)本會網站「求才訊息」內容由受本會補助之大專院校及研究機關(構)或其所屬之研究人員所刊登...
1,國立臺灣科技大學產學營運智財技轉中心-智權管理師一名(計畫人員),2025-07-22,https://www.nstc.gov.tw/folksonomy/detail/201c...,工作項目：\n1.工程、應科、人文學院下列案件業務辦理與收益分配\n專利授權、讓與、技術移轉...
2,臺中榮民總醫院 細胞治療與再生醫學中心實驗室 徵求專任研究助理2名,2025-07-22,https://www.nstc.gov.tw/folksonomy/detail/34c2...,【應徵要求】\n1. 工作經歷：不拘\n2. 國內、外大學生命科學、生物學、醫技系、醫藥衛生...
3,中國醫藥大學生化暨分子生物研究所 張瓊文老師實驗室博士後研究員,2025-07-22,https://www.nstc.gov.tw/folksonomy/detail/719b...,中國醫藥大學生化暨分子生物研究所 張瓊文老師實驗室博士後研究員\n【職缺名稱】國科會補助研究...
4,國立中正大學管理學院金融科技碩士學位學程誠徵專任教師,2025-07-22,https://www.nstc.gov.tw/folksonomy/detail/1c13...,一、徵聘職級及員額: 助理教授級(含)以上師資1名\n二、起聘日期: 民國 115年2月1日...
